<a href="https://colab.research.google.com/github/ved3prakas/Project/blob/master/IMG_SCRAPPING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MOUNT DRIVE AND CD TO ROOT DIRECTORY

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%pwd

'/content'

In [ ]:
import os
ROOT = "/content/drive/My Drive/Ineuron_DL/bus_truck" # Change path as per your drive
os.chdir(ROOT)
os.getcwd() # verify that you are in root folder

'/content/drive/My Drive/Ineuron_DL/bus_truck'

# INSTALL REQUIREMENTS

In [ ]:
!pip install selenium webdriver-manager
!apt-get update 
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 5.4MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [95.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:14 ht

In [ ]:
import os
import time
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

def fetch_image_urls(query: str, max_links_to_fetch: int, wd: webdriver, sleep_between_interactions: int = 1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

        # build the google query

    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)

        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls


def persist_image(folder_path:str,url:str, counter):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        f = open(os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg"), 'wb')
        f.write(image_content)
        f.close()
        print(f"SUCCESS - saved {url} - as {folder_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")




# def search_and_download(search_term: str, driver_path: str, target_path='./images', number_images=25):
def search_and_download(search_term: str, target_path='./images', number_images=10):

    target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' '))) # make the folder name inside images with the search string

    if not os.path.exists(target_folder):
        os.makedirs(target_folder) # make directory using the target path if it doesn't exist already

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    with webdriver.Chrome('chromedriver', chrome_options=chrome_options) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)

    counter = 0
    for elem in res:
        persist_image(target_folder, elem, counter)
        counter += 1

# def getData(CSV_PATH="/content/drive/My Drive/Ineuron_DL/bus_truck/List of class.csv", memberName=None):
#     sw_df = pd.read_csv(CSV_PATH)[['Name of class', "Names"]]
#     return sw_df.loc[sw_df["Names"] == memberName["Name of class"].to_list()]

def getSS(searchList =None, MAX_IMAGES=50):
  for search_term in searchList:
     start = time.perf_counter()
     print("##"*10, f"Searching for: {search_term}", "##"*10)
     search_and_download(search_term=search_term, number_images=MAX_IMAGES) # method to download images
     end = time.perf_counter()
     print(f"time taken: {end - start} seconds")
     print("##"*20)




# UPDATE YOUR MEMBER_NAME HERE

## Available names are-

['Amar', 'Ved', 'Arun', 'Sunny', 'Archana']

Choose accordingly

In [71]:

MEMBER_NAME = "Ved" # Change Name: ['Amar', 'Ved', 'Arun', 'Sunny', 'Archana']

#extract_SS_list = getData(memberName=MEMBER_NAME)

getSS(searchList=["Bus","Truck","Car","Auto"])

#################### Searching for: Bus ####################
Found: 100 search results. Extracting links from 0:100


KeyboardInterrupt: ignored

In [ ]:
import pandas
df=pd.read_csv("/content/drive/My Drive/Ineuron_DL/bus_truck/List of class.csv")

In [ ]:
df

,S.No,Name of class,Unnamed: 2,From,Names
0,0,Truck,NaN,Ved,Ved
1,1,Car,NaN,Ved,Ved
2,2,Bus,NaN,Ved,Ved
3,3,Auto,NaN,Ved,Ved


In [ ]:
df2=pd.read_csv("/content/drive/My Drive/Ineuron_DL/bus_truck/List of class.csv")[['Name of class', "Names"]]

In [ ]:
df2

,Name of class,Names
0,Truck,Ved
1,Car,Ved
2,Bus,Ved
3,Auto,Ved


In [44]:
df2.loc[df2["Names"]] == 'Ved'["Name of class"].index.to_list()


KeyError: ignored

In [ ]:
'Ved'[0]

'V'

In [46]:
df2["Names"]."S.No".to_list()

SyntaxError: ignored